- Aurora가 3개의 가용 영역 (AZ)에 걸쳐 6개의 쿼럼을 사용하며, 6개의 복사본 중 4개를 쓰기에 사용하고 6개의 복사본 중 3 개를 읽기/복구를 위해 사용한다는 점에 나타날 수 있는 문제를 살펴보고 있다.
  - 이전 글에서는 왜 6개가 필요한 최소 복사본 수인지, 쓰기와 읽기 작업 중 쿼럼의 성능 저하를 피하는 방법을 설명.
  - 많은 데이터 사본이 있으며 비용이 들어감.
  - Amazon Aurora의 스토리지 비용이 저렴하기 때문에 여러분들이 예상하는 대로 뭔가 특별한 것이 있다.

- 일반적으로 쓰기 세트가 구성 요소의 대부분을 이루고, 읽기 및 쓰기 세트가 중첩되는 집합을 쿼럼으로 설명.
- 기본 요구 사항은 단지 읽기 및 쓰기 세트가 전체 쿼럼 구성원 집합의 하위 집합이며, 모든 적합한 쓰기 하위 집합은 최소한 하나의 구성원이 적합한 읽기 하위 집합에 포함되어야 하고, 각 쓰기 하위 집합은 이전 버전의 쓰기 하위 집합과 중첩되야 함.
  - 차이점은 쿼럼 구성원들이 서로 동일해야 한다는 요구 사항이 없다는 것.
- 지연 시간, 비용 또는 내구성이 서로 다른 쿼럼 하위 집합을 혼합하여 쿼럼 세트를 구성 할 수 있다.
- 그런 다음 부울 논리(Boolean logic) 규칙을 사용하여 전체 쿼럼에 대한 쿼럼 구성원 요구 사항을 충족시키기 위해, 하위 집합 전체에 걸쳐 보다 정교한 읽기 및 쓰기 규칙을 만들 수 있다.

<br>

**비용을 줄이기 위한 Aurora의 운용방식**

- 쿼럼의 크기를 정하는 방법, 읽기 및 쓰기 증폭에 따른 문제를 피하는 방법과 함께 비용을 제어하는 방법에 대해 살펴 봄.

**데이터의 풀 세그먼트와 테일 세그먼트 혼합**

- Aurora에서 데이터베이스 볼륨은 10GB의 데이터 세그먼트로 구성.
  - 이러한 세그먼트는 보호 그룹으로 복제되며 6개의 복사본이 3개의 가용 영역(AZ)에 분산되어 있습니다.
  - 그러나 6개의 복사본이 모두 동일하지는 않음.
  - 복사본의 절반은 볼륨의 10GB 부분에 대한 데이터 페이지와 로그 레코드를 모두 포함하는 풀 세그먼트(Full Segment).
  - 나머지 절반은 로그 레코드만 포함하는 테일 세그먼트(Tail Segment).
  - 각 AZ에는 하나의 풀 세그먼트와 하나의 테일 세그먼트를 가지게 됨.

- 대부분의 데이터베이스는 실행 로그(Redo log) 스토리지보다 훨씬 많은 데이터 블록 스토리지를 가지고 있다.
  - 풀 세그먼트와 테일 세그먼트를 함께 사용하면 Aurora의 물리적 스토리지 요구 사항은 데이터베이스 크기의 6배였던 것이 3배가 약간 넘는 수준으로 낮아짐
  - “AZ + 1” 장애 사례를 견딜 수 있도록 설계된 시스템의 경우, 이는 사용자가 가질 수 있는 최소한의 복제 요소.

- 풀 세그먼트와 테일 세그먼트를 혼합하여 사용하면 읽기 및 쓰기 세트를 구성하는 방법이 바뀜.
  - 부울 논리 규칙을 사용하여 부분집합 간의 중첩을 보장하고, 임의의 복잡한 구성원 분포를 정확하게 수행 할 수 있다.
  - Aurora의 쓰기 쿼럼은 6개의 풀 및 테일 세그먼트 중 4개 또는(OR) 3개의 풀 세그먼트 중 3개로 구성.
  - 읽기 쿼럼은 6개의 풀 및 테일 세그먼트 중 3개이면서(AND) 3개 중 1개는 풀 세그먼트로 구성.


- 풀 세그먼트의 중첩 뿐만 아니라 쿼럼의 모든 세그먼트에 중첩이 있음을 알 수 있다.
  - 이를 통해 6 개의 세그먼트 중 4 개의 세그먼트에 로그 레코드를 반영.
  - 이들 중 적어도 하나는 풀 세그먼트이며 데이터 페이지를 생성.
  - 세 번째 글에서 설명한 대로 쿼럼 읽기를 피하기 위한 최적화 구조를 통해, 필요한 데이터가 위치한 스토리지 노드 한 곳의 풀 세그먼트에서 데이터를 조회.

- 장애가 발생한 세그먼트의 재생성(Rebuild) 및 손상된 쿼럼을 복구하기 위해 읽기 쿼럼을 사용.
  - 데이터베이스 쓰기 마스터 노드를 다시 시작해야 할 경우 로컬 상태를 재생성하는데 사용.
  - 테일 세그먼트 중 하나에 오류가 발생하면 단순한 쿼럼 모델과 마찬가지로 다른 3개 복사본 중 하나로부터 쉽게 복구.

- 만약 풀 세그먼트 중 하나가 하나에 오류가 발생하면 조금 더 복잡해 짐.
  - 오류가 발생한 세그먼트가 쓰기작업의 일부로 작성한 복사본 일 수 있다.
  - 이 경우 또 다른 풀 세그먼트가 있을 뿐더러, 최신 풀 세그먼트를 재생성할 수 있는 충분한 실행로그가 있다.
  - 또한 쿼럼 내 세그먼트간 통신을 통해 누락된 쓰기가 없는지 확인하여, 전체 쓰기 작업을 지연 시킬 수 있는 풀 세그먼트 재생성 작업을 최소화.


**서로 다른 구성원의 쿼럼 세트로 비용 관리**

- 서로 다른 구성원의 쿼럼 세트를 사용하는 것은 비용을 절감하는 좋은 방법.
  - 다양한 옵션을 사용할 수 있습니다.
  - 낮은 지연 시간을 위한 로컬 디스크와 내구성 / 가용성을 위한 원격 디스크를 결합한 쿼럼을 사용할 수 있습니다.
  - 성능과 처리량을 위한 SSD와 저비용 복제본을 통해 내구성을 향상 시킨 HDD를 결합한 쿼럼을 사용할 수 있습니다.
  - 재해 복구 능력을 향상 시키기 위해 AWS 리전간 데이터를 결합하는 쿼럼이 있을 수 있습니다.
  - 좋은 결과를 얻기 위해 사용자가 조절할 필요가 있는 부분이 많지만, 그에 대한 대가가 상당할 수 있습니다.
  - Aurora의 경우 이전에 설명한 쿼럼 세트 모델을 사용하면 높은 내구성과 가용성 및 성능을 제공하는 동시에 저렴함 스토리지 비용을 달성할 수 있습니다.
